In [2]:
%pip install openpyxl


In [22]:
import pandas as pd
import cv2
from PIL import Image, ImageDraw
import os


def load_data(file_path: str, variant: int):
    """
    Load data from an Excel file and retrieve data for the given variant.
    """
    df = pd.read_excel(file_path)
    print(f"Loaded DataFrame:\n{df}")
    
    # Assuming variant is an index in the DataFrame
    data = df.iloc[variant]  # Replace with correct indexing logic if needed
    print(f"Data for Variant {variant}:\n{data}")
    return data


def detect_face_and_eyes(image_path: str):
    """
    Detect face and eyes in an image using Haar Cascade.
    """
    # Load the cascades
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
    
    # Read the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces) == 0:
        raise ValueError("No faces detected in the image.")
    
    # Assume the first detected face is the target
    x, y, w, h = faces[0]
    face = img[y:y+h, x:x+w]
    
    # Detect eyes within the face region
    face_gray = gray[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(face_gray)
    if len(eyes) < 2:
        raise ValueError("Could not detect both eyes in the face.")
    
    return img, (x, y, w, h), eyes[:2]  # Return the first two detected eyes


def add_glasses(image, face_coords, eyes, color=(252, 3, 11), thickness=1):
    """
    Add circular glasses to an image based on detected face and eye coordinates.
    """
    x, y, w, h = face_coords
    eye1, eye2 = eyes
    
    # Unpack eye coordinates (relative to the face)
    ex1, ey1, ew1, eh1 = eye1
    ex2, ey2, ew2, eh2 = eye2
    
    # Calculate eye centers (relative to the original image)
    eye1_center = (x + ex1 + ew1 // 2, y + ey1 + eh1 // 2)
    eye2_center = (x + ex2 + ew2 // 2, y + ey2 + eh2 // 2)
    
    # Calculate radius of the glasses based on eye dimensions
    radius = max(ew1, eh1, ew2, eh2) // 3 + 10  # Add padding for the glasses
    
    # Draw glasses
    cv2.circle(image, eye1_center, radius, color, thickness)
    cv2.circle(image, eye2_center, radius, color, thickness)
    
    # Draw bridge (line between the glasses)
    line_start = (eye1_center[0] + radius, eye1_center[1])
    line_end = (eye2_center[0] - radius, eye2_center[1])
    cv2.line(image, line_start, line_end, color, thickness // 2)
    
    # Optionally draw arms of the glasses
    cv2.line(image, (eye1_center[0] - radius, eye1_center[1]), (eye1_center[0] - radius - 30, eye1_center[1]), color, thickness // 2)
    cv2.line(image, (eye2_center[0] + radius, eye2_center[1]), (eye2_center[0] + radius + 30, eye2_center[1]), color, thickness // 2)
    
    return image


def process_image(data, image_path, output_path, color=(252, 3, 11), thickness=5, resize_dim=(600, 600)):
    """
    Process the image: detect face, add glasses, center face, and resize.
    """
    # Detect face and eyes
    img, face_coords, eyes = detect_face_and_eyes(image_path)
    
    # Add glasses
    img_with_glasses = add_glasses(img, face_coords, eyes, color=color, thickness=thickness)
    
    # Crop to center the face
    x, y, w, h = face_coords
    face_centered = img_with_glasses[max(0, y-h//2):y+h+h//2, max(0, x-w//2):x+w+w//2]
    
    # Resize to specified dimensions
    resized = cv2.resize(face_centered, resize_dim)
    
    # Save the processed image
    cv2.imwrite(output_path, resized)
    print(f"Processed image saved to: {output_path}")


# Example usage
variant = 2  # Replace with your variant
data = load_data("lab6.xlsx", variant)

image_path = "images/longbottom.jpg"  # Replace with your input image path
output_path = "output-white-slim.jpg"

process_image(data, image_path, output_path, color=(255, 255, 255), thickness=4)


Loaded DataFrame:
   N         file name image size glasses color  line width
0  1  emma-watson2.jpg    300x300           red           2
1  2   emma-watson.jpg    400x400          blue           3
2  3         draco.jpg    700x700          cian           4
3  4    longbottom.jpg    600x600       magenta           5
4  5    ron_wesley.jpg    500x500        yellow           6
Data for Variant 2:
N                        3
file name        draco.jpg
image size         700x700
glasses color         cian
line width               4
Name: 2, dtype: object
Processed image saved to: output-white-slim.jpg
